## Install Libraries

In [182]:
%pip install pandas requests --quiet

Note: you may need to restart the kernel to use updated packages.


## Connect to database

In [185]:
%%sql
USE winter_wikipedia;
select count(8) from my_book;

count(8)
40


## Read the file into Pandas frame

In [187]:
import pandas as pd
import requests
from io import StringIO

# Provide the URL of your CSV file
url = "https://raw.githubusercontent.com/madhukarkumar/my_csvs/main/ratings_small.csv"

# Use requests to get the content of the CSV file
content = requests.get(url).text

# Create a pandas DataFrame
df = pd.read_csv(StringIO(content))

# Show the first 5 rows of the dataframe
print(df.head())

   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205


In [188]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

## Create table based on the columns of the CSV file

In [191]:

# Generate a CREATE TABLE statement based on the DataFrame's dtypes
create_table_statement = "CREATE TABLE IF NOT EXISTS csv_table ("

for col, dtype in df.dtypes.items():
    if "int" in str(dtype):
        sql_type = "INT"
    elif "float" in str(dtype):
        sql_type = "FLOAT"
    elif "object" in str(dtype):
        sql_type = "TEXT"
    else:
        sql_type = "TEXT"
    create_table_statement += "\n{} {},".format(col, sql_type)

create_table_statement = create_table_statement.rstrip(",") + "\n);"

# Execute the CREATE TABLE statement in the database
db_connection.execute(create_table_statement)


## Load the data into the CSV file

In [ ]:
# Insert the data row by row
for i, row in df.iterrows():
    placeholders = ', '.join(['%s'] * len(row))
    columns = ', '.join(row.keys())
    sql = "INSERT INTO csv_table ( %s ) VALUES ( %s )" % (columns, placeholders)
    db_connection.execute(sql, tuple(row))
